In [1]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install langdetect
!pip install polars
!pip install groq
!pip install pyarrow
!pip install fastparquet

  Cloning https://github.com/boudinfl/pke.git to c:\users\xavmo\appdata\local\temp\pip-req-build-i1zp42f6
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63

  Running command git clone -q https://github.com/boudinfl/pke.git 'C:\Users\xavmo\AppData\Local\Temp\pip-req-build-i1zp42f6'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.



  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6161142 sha256=3ce9f69aa1d4e248daa516120f6119948394f6501b2427d7697449fca896b064
  Stored in directory: C:\Users\xavmo\AppData\Local\Temp\pip-ephem-wheel-cache-bmrn1f1u\wheels\d5\46\97\85535b5b449f70b6a3c8d1138ce8587345876891e25bfe7954
Successfully built pke
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4
  Attempting uninstall: smart-open
    Found existing installation: smart-open 5.1.0
    Uninstalling smart-open-5.1.0:
      Successfully uninstalled smart

In [42]:
!pip install --upgrade pandas

In [3]:
# Librairies nécessaires
import numpy as np
import requests
import pandas as pd
import polars as pl
from langdetect import detect
import json
import tqdm
from tqdm import trange

import unicodedata
import re

## Importation d'un corpus : 

Sorbonne Université à tous son corpus de publications principalement sur HAL.  
Cependant, suite à une annonce de partenariat avec OpenAlex et allant de plus en plus sur cette bibliothèque gratuite et open source, le choix d'aller chercher dans un premier temps mon corpus dessus est donc le plus pertinant.  
Dans un premier temps, il nous faut donc comparer la différence des mots clés utilisés dans HAL et OpenAlex via le travail de Nacef effectué sur le github suivant (https://github.com/obtic-sorbonne/keywords/blob/main/TER_Keyword_Extraction_sur_BDD_HAL.ipynb).  
Regardons un premier temps quelles sont les métadonnées des publications sur HAL et OA en prenant pour exemple une publication de Stephane Le Crom

In [4]:
import requests
import json

hal_id = 'hal-00122816'
url = f"https://api.archives-ouvertes.fr/search/?q=halId_s:{hal_id}&wt=json"

response = requests.get(url)
# Faire la requête à l'API
if response.status_code == 200:
    
    # Conversion de la réponse en JSON
    data = json.loads(response.text)
    # Prendre le premier document (s'il y en a plusieurs)
    doc = data['response']['docs'][0]
    print(doc)
    # Vérifier s'il y a des documents dans la réponse
    if data['response']['numFound'] > 0:
        
        # Afficher les noms de toutes les métadonnées
        for key in doc.keys():
            print(key)
    else:
        print("Aucune publication trouvée avec cet identifiant HAL.")
else:
    print(f"Erreur lors de la requête : {response.status_code}")

{'docid': '122816', 'label_s': 'Marika Kapsimali, Stéphane Le Crom, Philippe Vernier. A natural history of vertebrate dopamine receptors.. Anita Sidhu, Marc Laruelle, Philippe Vernier. Dopamine Receptors and Transporters, Marcel Dekker Inc, pp.1-45, 2003. &#x27E8;hal-00122816&#x27E9;', 'uri_s': 'https://hal.science/hal-00122816'}
docid
label_s
uri_s


In [5]:
# ID de l'auteur
author_id = "a5007814380"  

# URL de l'API
url = f"https://api.openalex.org/works?filter=author.id:{author_id}"

# Faire la requête à l'API
response = requests.get(url)

# Vérifier que la requête a réussi
if response.status_code == 200:
    
    # Conversion de la réponse en JSON
    data = json.loads(response.text)
    print(data.keys())
    i=1
    for work in data['results'] : 
        if i==1 :
            print(work.keys())
            i+=1

dict_keys(['meta', 'results', 'group_by'])
dict_keys(['id', 'doi', 'title', 'display_name', 'publication_year', 'publication_date', 'ids', 'language', 'primary_location', 'type', 'type_crossref', 'indexed_in', 'open_access', 'authorships', 'countries_distinct_count', 'institutions_distinct_count', 'corresponding_author_ids', 'corresponding_institution_ids', 'apc_list', 'apc_paid', 'fwci', 'has_fulltext', 'cited_by_count', 'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext', 'primary_topic', 'topics', 'keywords', 'concepts', 'mesh', 'locations_count', 'locations', 'best_oa_location', 'sustainable_development_goals', 'grants', 'datasets', 'versions', 'referenced_works_count', 'referenced_works', 'related_works', 'ngrams_url', 'abstract_inverted_index', 'cited_by_api_url', 'counts_by_year', 'updated_date', 'created_date'])


  On voit qu'il nous ait bien compliqué d'obtenir par code tous les paramètres des métadonnées d'une publication sur HAL. Je vous encourage donc à suivre
le lien suivant afin de mieux comprendre les différents paramètres renseignés initialement (https://hal.science/help/information).  
Nous remarquons à présent avec la seconde brique de code tous les différents paramètres renseignés par les métadonnées d'une publication d'OpenAlex. Celles-ci sont nettement plus complexes et intéressantes à utiliser que celles de HAL, et ce pour de multiples raisons :  
- Keywords et scorings : 
- Topics : 
- Citations :  

Dans un premier temps, une étude des keywords en fonction d'HAL et d'OpenAlex sera faite afin de verifier la pertinence de leurs attributions. Pour ce
faire, nous appliquerons un raisonnement par induction. Nous sélectionnerons un plus petit corpus constitué des travaux de Stéphane Le Crom afin d'obtenir des premiers résultats rapides. Puis, nous appliquerons notre protocole sur notre corpus global pour une comparaison à plus grande échelle.

### Quel protocole utiliser ?

In [6]:
# test a deux lvl pour keywords -> mots clés exacts similaires ou etymologiquement similaire (via du LLM ou scoring ??)
#                               -> comparaison par topic de ses mots clés d'open alex (Comparaison avec les autres topics??)
#                               -> comment calculer la similarité?

### Récupération des publications de Stéphane Le Crom sur HAL et OpenAlex

In [7]:
auteurs = ["Stéphane Le Crom"]

# Initialisez une liste vide pour stocker les métadonnées des publications
publications = []

# URL de base de l'API HAL
base_url = "https://api.archives-ouvertes.fr/search/"

# Parcourez la liste des auteurs et récupérez leurs publications
for auteur in auteurs:
    params = {
        "q": f'authFullName_s:"{auteur}"',
        "fl": "authFullName_s,authIdHal_i,authIdHal_s,title_s,halId_s,producedDateY_i,doiId_s,abstract_s,uri_s,domain_s,keyword_s",
        "rows": 10000  # Augmentez le nombre de lignes si nécessaire
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        publications.extend(data["response"]["docs"])
    else:
        print(f"Erreur pour l'auteur {auteur}: {response.status_code} - {response.text}")

# Créez un DataFrame Pandas à partir des métadonnées des publications
column_order = ["authFullName_s", "authIdHal_i", "authIdHal_s", "title_s", "halId_s", "producedDateY_i", "doiId_s", "keyword_s", "abstract_s", "uri_s", "domain_s"]

df = pd.DataFrame(publications, columns= column_order)

# Sauvegardez le DataFrame dans un fichier CSV
df.to_csv("publications_hal_stephane_le_crom.csv", index=False)

# Affichez le nombre total de publications récupérées
print(f"Nombre total de publications récupérées : {len(df)}")

Nombre total de publications récupérées : 64


In [8]:
# Conversion les listes en chaînes dans la colonne "title_s"
df["title_s"] = df["title_s"].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Suppression des doublons à base du titre seulement
df = df.drop_duplicates(subset="title_s")
print(f"Publications restantes après nettoyage : {len(df)}")

Publications restantes après nettoyage : 62


In [9]:
# ID de l'auteur
author_id = "a5052835091"

# URL de l'API
url = f"https://api.openalex.org/works?filter=author.id:{author_id}"

# Faire la requête à l'API
response = requests.get(url)

# Vérifier que la requête a réussi
if response.status_code == 200:
    # Conversion de la réponse en JSON
    data = json.loads(response.text)

    # Sélection les métadonnées précises
    selected_data = []
    
    for work in data['results']:
        
        author_ids = []
        if 'authorships' in work: 
            for authorship in work['authorships']:
                # Ajoutez l'identifiant de l'auteur à la liste 'author_ids'
                author_ids.append(authorship["author"]["id"])
                
        keywords, scorings = [], [] 
        if 'keywords' in work:
            for id in work['keywords']:
                keywords.append(id['display_name'])
                scorings.append(id['score'])
                
        selected_data.append({
            'id' : work['id'],
            'doi': work['doi'],
            'title': work['title'],
            'authors_id' : author_ids, 
            'keywords' : keywords, 
            'scorings' : scorings,
            'referenced_works': work['referenced_works'], 
            'related_works': work['related_works'],
            'cited_by_api_url': work['cited_by_api_url'],    
        })

    # Conversion des données sélectionnées en DataFrame
    df2 = pd.DataFrame(selected_data)

    # Exportation du DataFrame en CSV
    df2.to_csv('publications_oa_stephane_le_crom.csv', index=False)
else:
    print(f"Erreur : {response.status_code}")

In [16]:
def name_to_keep_ind(groups):
    """
    Function to determine if a text should be kept or not.

    Input:
    groups: list of character groups

    Output:
    0: if text should be not used
    1: if text should be used
    """
    # Groups of characters that do not perform well
    groups_to_skip = ['HIRAGANA', 'CJK', 'KATAKANA','ARABIC', 'HANGUL', 'THAI','DEVANAGARI','BENGALI',
                      'THAANA','GUJARATI','CYRILLIC']
    
    if any(x in groups_to_skip for x in groups):
        return 0
    else:
        return 1

def remove_non_latin_characters(text):
    """
    Function to remove non-latin characters.

    Input:
    text: string of characters

    Output:
    final_char: string of characters with non-latin characters removed
    """
    final_char = []
    groups_to_skip = ['HIRAGANA', 'CJK', 'KATAKANA','ARABIC', 'HANGUL', 'THAI','DEVANAGARI','BENGALI',
                      'THAANA','GUJARATI','CYRILLIC']
    for char in text:
        try:
            script = unicodedata.name(char).split(" ")[0]
            if script not in groups_to_skip:
                final_char.append(char)
        except:
            pass
    return "".join(final_char)
    
def group_non_latin_characters(text):
    """
    Function to group non-latin characters and return the number of latin characters.

    Input:
    text: string of characters

    Output:
    groups: list of character groups
    latin_chars: number of latin characters
    """
    groups = []
    latin_chars = []
    text = text.replace(".", "").replace(" ", "")
    for char in text:
        try:
            script = unicodedata.name(char).split(" ")[0]
            if script == 'LATIN':
                latin_chars.append(script)
            else:
                if script not in groups:
                    groups.append(script)
        except:
            if "UNK" not in groups:
                groups.append("UNK")
    return groups, len(latin_chars)

def check_for_non_latin_characters(text):
    """
    Function to check if non-latin characters are dominant in a text.

    Input:
    text: string of characters

    Output:
    0: if text should be not used
    1: if text should be used
    """
    groups, latin_chars = group_non_latin_characters(str(text))
    if name_to_keep_ind(groups) == 1:
        return 1
    elif latin_chars > 20:
        return 1
    else:
        return 0

def clean_title(old_title):
    """
    Function to check if title should be kept and then remove non-latin characters. Also
    removes some HTML tags from the title.
    
    Input:
    old_title: string of title
    
    Output:
    new_title: string of title with non-latin characters and HTML tags removed
    """
    keep_title = check_for_non_latin_characters(old_title)
    if (keep_title == 1) & isinstance(old_title, str):
        new_title = remove_non_latin_characters(old_title)
        if '<' in new_title:
            new_title = new_title.replace("<i>", "").replace("</i>","")\
                                 .replace("<sub>", "").replace("</sub>","") \
                                 .replace("<sup>", "").replace("</sup>","") \
                                 .replace("<em>", "").replace("</em>","") \
                                 .replace("<b>", "").replace("</b>","") \
                                 .replace("<I>", "").replace("</I>", "") \
                                 .replace("<SUB>", "").replace("</SUB>", "") \
                                 .replace("<scp>", "").replace("</scp>", "") \
                                 .replace("<font>", "").replace("</font>", "") \
                                 .replace("<inf>","").replace("</inf>", "") \
                                 .replace("<i /> ", "") \
                                 .replace("<p>", "").replace("</p>","") \
                                 .replace("<![CDATA[<B>", "").replace("</B>]]>", "") \
                                 .replace("<italic>", "").replace("</italic>","")\
                                 .replace("<title>", "").replace("</title>", "") \
                                 .replace("<br>", "").replace("</br>","").replace("<br/>","") \
                                 .replace("<B>", "").replace("</B>", "") \
                                 .replace("<em>", "").replace("</em>", "") \
                                 .replace("<BR>", "").replace("</BR>", "") \
                                 .replace("<title>", "").replace("</title>", "") \
                                 .replace("<strong>", "").replace("</strong>", "") \
                                 .replace("<formula>", "").replace("</formula>", "") \
                                 .replace("<roman>", "").replace("</roman>", "") \
                                 .replace("<SUP>", "").replace("</SUP>", "") \
                                 .replace("<SSUP>", "").replace("</SSUP>", "") \
                                 .replace("<sc>", "").replace("</sc>", "") \
                                 .replace("<subtitle>", "").replace("</subtitle>", "") \
                                 .replace("<emph/>", "").replace("<emph>", "").replace("</emph>", "") \
                                 .replace("""<p class="Body">""", "") \
                                 .replace("<TITLE>", "").replace("</TITLE>", "") \
                                 .replace("<sub />", "").replace("<sub/>", "") \
                                 .replace("<mi>", "").replace("</mi>", "") \
                                 .replace("<bold>", "").replace("</bold>", "") \
                                 .replace("<mtext>", "").replace("</mtext>", "") \
                                 .replace("<msub>", "").replace("</msub>", "") \
                                 .replace("<mrow>", "").replace("</mrow>", "") \
                                 .replace("</mfenced>", "").replace("</math>", "")

            if '<mml' in new_title:
                all_parts = [x for y in [i.split("mml:math>") for i in new_title.split("<mml:math")] for x in y if x]
                final_parts = []
                for part in all_parts:
                    if re.search(r"\>[$%#!^*\w.,/()+-]*\<", part):
                        pull_out = re.findall(r"\>[$%#!^*\w.,/()+-]*\<", part)
                        final_pieces = []
                        for piece in pull_out:
                            final_pieces.append(piece.replace(">", "").replace("<", ""))
                        
                        final_parts.append(" "+ "".join(final_pieces) + " ")
                    else:
                        final_parts.append(part)
                
                new_title = "".join(final_parts).strip()
            else:
                pass

            if '<xref' in new_title:
                new_title = re.sub(r"\<xref[^/]*\/xref\>", "", new_title)

            if '<inline-formula' in new_title:
                new_title = re.sub(r"\<inline-formula[^/]*\/inline-formula\>", "", new_title)

            if '<title' in new_title:
                new_title = re.sub(r"\<title[^/]*\/title\>", "", new_title)

            if '<p class=' in new_title:
                new_title = re.sub(r"\<p class=[^>]*\>", "", new_title)
            
            if '<span class=' in new_title:
                new_title = re.sub(r"\<span class=[^>]*\>", "", new_title)

            if 'mfenced open' in new_title:
                new_title = re.sub(r"\<mfenced open=[^>]*\>", "", new_title)
            
            if 'math xmlns' in new_title:
                new_title = re.sub(r"\<math xmlns=[^>]*\>", "", new_title)

        if '<' in new_title:
            new_title = new_title.replace(">i<", "").replace(">/i<", "") \
                                 .replace(">b<", "").replace(">/b<", "") \
                                 .replace("<inline-formula>", "").replace("</inline-formula>","")
        if new_title.isupper():
            new_title = new_title.title()
        
        return new_title
    else:
        return ''
    
def liaison_second(df1_hal, df2_oa):
    # On clean les titres de la df d'OpenAlex
    for i, title in enumerate(df2_oa['title']):
        new_title = clean_title(title)  # clean_title : la fonction de nettoyage appropriée
        df2_oa.loc[i, 'title'] = new_title

    # On convertit les titres d'oa en liste
    list_oa_title = df2_oa['title'].tolist()

    # On supprime toutes les publications qui ne sont pas en commun
    indexes_to_drop = []
    for i, title in enumerate(df1_hal['title_s']):
        if title not in list_oa_title:
            indexes_to_drop.append(i)

    df1_hal.drop(indexes_to_drop, inplace=True)
    
def clean_doi_oa(old_doi):
    return old_doi.replace("https://doi.org/", "")

def liaison(df1_hal, df2_oa):
    # On clean les titres de la df d'OpenAlex
    for i, doi in enumerate(df2_oa['doi']):
        new_doi = clean_doi_oa(doi)
        df2_oa.loc[i, 'doi'] = new_doi

    # On convertit les titres d'oa en liste
    list_hal_title = df1_hal['doiId_s'].tolist()
    df2_change = pd.DataFrame()

    # On supprime toutes les publications qui ne sont pas en commun
    indexes_to_drop = []
    
    for i, doi in enumerate(df2_oa['doi']):
        if doi in list_hal_title:
            #df2_change = df2_change.append(df1_hal.iloc[i])
            df2_change = pd.concat([df2_change, df1_hal.iloc[[i]]])
        else : 
            indexes_to_drop.append(i)
            
    df2_oa.drop(indexes_to_drop, inplace=True)
    df2_change.to_csv('pub_hal_liees.csv', index=False)
    
    return df2_change

In [17]:
df2_change = liaison(df,df2)
print(len(df2_change))
print(len(df2))

25
25


In [33]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = "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"

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

def extract_keywords_groq(text, models):
    prompt = f"As a keyword extraction master, your only mission here is to extract only the most relevent keywords that are present in the text. Put the list of keywords between brackets, comma-seperated.DO NOT write something else than the keywords you're supposed to extract from the text. Skip the preamble and provide only the best keywords (present in the text). The text:\n{text} VERY IMPORTANT : Put the list of keywords between brackets, comma-seperated like this ['keyword1', 'keyword2, 'keyword3' etc...] NO TICKS only what ive said"
    chat_completion = client.chat.completions.create(
    messages=[

            {
                "role": "user",
                "content": prompt,
            },
            {
                "role": "assistant",
                "content": "["
            }
        ],
        model=models,
        max_tokens=1024
    )
    response = chat_completion.choices[0].message.content
    keyword_list = [keyword.strip('"') for keyword in response.strip("[]").split(', ')]


    return keyword_list

def extract_groq(df, list_models, use_title = False):
    
    for model in list_models :

        list_kw = []

        for row in trange(len(df)):
            text = df.loc[row,'abstract_s']
            if pd.isnull(text):
                text = ''
            else:
                text = str(text)
                
            if use_title:
                title = df.loc[row, 'title_s']
                if not pd.isnull(title):
                    text += str(title)
                    
            kw = extract_keywords_groq(text, model)
            list_kw.append(kw)
            
        list_of_dicts = [{'list of kw': keywords} for keywords in list_kw]
        df_kw = pl.DataFrame(list_of_dicts)

        df[model] = df_kw['list of kw']
    
    return df


In [34]:
models = ['llama3-8b-8192',
          #'gemma-7b-it',
          "llama3-70b-8192"]

df = pd.read_csv('pub_hal_liees.csv')

df_llm = extract_groq(df, models)
df_llm_title = extract_groq(df_llm, models, use_title = True)


100%|██████████| 25/25 [01:34<00:00,  3.79s/it]


In [35]:
df_llm_title.to_csv('pub_hal_liees_keywords.csv', index=True)

In [36]:
df_llm_title.head()

,authFullName_s,authIdHal_i,authIdHal_s,title_s,halId_s,producedDateY_i,doiId_s,keyword_s,abstract_s,uri_s,domain_s,llama3-8b-8192,llama3-70b-8192
0,"['Thomas Portnoy', 'Antoine Margeot', 'Verena ...","[919327, 937936, 13760, 919328, 919335, 919336]",['stephane-le-crom'],Differential regulation of the cellulase trans...,hal-02879336,2011,10.1128/EC.00208-10,NaN,['Due to its capacity to produce large amounts...,https://hal.sorbonne-universite.fr/hal-02879336,"['0.sdv', '1.sdv.bbm', '2.sdv.bbm.gtp', '0.sdv...","[Trichoderma reesei, cellulases, biofuel, lign...","[Trichoderma reesei, cellulase, biofuel, ligno..."
1,"['Fanny Coulpier', 'Stéphane Le Crom', 'Gérald...",[13760],['stephane-le-crom'],Novel features of boundary cap cells revealed ...,hal-02879350,2009,10.1002/glia.20862,NaN,"['Neural crest (NC) cells are a multipotent, h...",https://hal.sorbonne-universite.fr/hal-02879350,"['0.sdv', '1.sdv.bbm', '2.sdv.bbm.gtp', '0.sdv...","[Neural crest, boundary cap, Schwann cells, pe...","[Neural crest, Peripheral nervous system, Glia..."
2,"['Nathalie Lehmann', 'Sandrine Perrin', 'Clair...","[1148850, 174545, 13760, 748529, 185045]","['charlotte-berthelier', 'celine-hernandez', '...",Eoulsan 2: an efficient workflow manager for r...,hal-03784195,2021,10.1101/2021.10.13.464219,NaN,['A bstract Motivation Core sequencing facilit...,https://hal.science/hal-03784195,"['0.sdv', '1.sdv.bibs']","[Eoulsan, workflow engine, sequencing data, tr...","[Eoulsan, workflow, sequencing, RNA-seq, trans..."
3,"['Andrea Sartirana', 'Laurent Jourdren', 'Paul...","[950950, 185045, 950951, 1228770, 741166, 13760]","['laurent-jourdren', 'david-chamont', 'phbusso...",Eoulsan : analyse du séquençage à haut débit d...,hal-00927507,2013,NaN,NaN,['Eoulsan : analyse du séquençage à haut débit...,https://hal.science/hal-00927507,"['0.info', '1.info.info-dc']","[Eoulsan, séquençage, débit, cloud, grille]","[Eoulsan, analyse, séquençage, haut débit, clo..."
4,"['Sophie Lemoine', 'Florence Combes', 'Nicolas...","[908599, 918596, 13760]","['lemoine-sophie', 'stephane-le-crom']",Goulphar: rapid access and expertise for stand...,inserm-00122139,2006,10.1186/1471-2105-7-467,NaN,['BACKGROUND: Raw data normalization is a crit...,https://inserm.hal.science/inserm-00122139,"['0.sdv', '1.sdv.bibs']","[raw data normalization, microarray data, R/Bi...","[microarray, normalization, R, BioConductor, d..."


//////////////////////////////////////////////////////////////////////////////////////////////////////////////////  
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////  
////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [23]:
# Script de Waly modifié.
# Liste des noms d'auteurs que vous souhaitez rechercher
auteurs = [
    "Anne-Virginie Salsac",
    "Dan Istrate",
    "Eric Leclerc",
    "Julien \"Le Duigou\"",
    "Marie-Christine \"Ho Ba Tho\"",
    "Sofiane Boudaoud",
    "Glenn Roe",
    "Motasem ALRAHABI",
    "Arnaud Latil",
    "Christian NERI",
    "Clément Mabi",
    "David Flacher",
    "M. Shawky",
    "Serge Bouchardon",
    "Harry Sokol",
    "Bérangère Bihan-Avalle",
    "Caroline Marti",
    "Laurent Petit",
    "Pierre-Carl Langlais",
    "David Klatzmann",
    "Raphael Gavazzi",
    "Benjamin Wandelt",
    "christophe pichon",
    "Guilhem Lavaux",
    "Henry Joy McCracken",
    "Kumiko Kotera",
    "Yohan Dubois",
    "A. Marco Saitta",
    "Dirk Stratmann",
    "Guillaume Ferlat",
    "Slavica Jonic",
    "Alex Chin",
    "Fabrice Carrat",
    "Pierre-Yves Boëlle",
    "Renaud Piarroux",
    "Christophe Guillotel-Nothmann",
    "Jean-Marc Chouvel",
    "Nicolas Obin",
    "Philippe Esling",
    "Alexandre Coninx",
    "Baptiste Caramiaux",
    "Benjamin Piwowarski",
    "Catherine Achard",
    "Catherine Pelachaud",
    "Gilles Bailly",
    "Jérôme Szewczyk",
    "Kevin Bailly",
    "Laure Soulier",
    "Marie-Aude Vitrani",
    "Matthieu Cord",
    "Mehdi Khamassi",
    "Mohamed CHETOUANI",
    "Nathanaël Jarrassé",
    "Nicolas Bredeche",
    "Nicolas Perrin-Gilbert",
    "Nicolas Thome",
    "Olivier Schwander",
    "Olivier Sigaud",
    "Pascal Morin",
    "Pierre Bessière",
    "Sinan Haliyo",
    "Stéphane Doncieux",
    "Alessandra Carbone",
    "Elodie Laine",
    "Martin Weigt",
    "Benoit Semelin",
    "Emeric Bron",
    "Emmanuel Bertin",
    "Françoise Combes",
    "Maryvonne Gerin",
    "Philippe Salomé",
    "Baptiste Cecconi",
    "Philippe Zarka",
    "Ferdinand Dhombres",
    "Jean Charlet",
    "Xavier Tannier",
    "Amal \"El Fallah Seghrouchni\"",
    "Andrea Pinna",
    "Antoine Miné",
    "Béatrice Bérard",
    "Bertrand Granado",
    "Bruno Escoffier",
    "Carola Doerr",
    "Christoph Dürr",
    "Christophe Denis",
    "Christophe Marsala",
    "Colette Faucher",
    "Emmanuel HYON",
    "Emmanuelle Encrenaz-Tiphène",
    "Evripidis Bampis",
    "Fanny Pascual",
    "Haralampos Stratigopoulos",
    "Jean-Daniel Kant",
    "Jean-Gabriel Ganascia",
    "Jean-Noël Vittaut",
    "Lionel Tabourier",
    "Maria Potop-Butucaru",
    "Matthieu Latapy",
    "Nicolas MAUDET",
    "Olivier Spanjaard",
    "Patrice Perny",
    "Patrick Gallinari",
    "Sébastien Tixeuil",
    "Spyros Angelopoulos",
    "Stéphane Gançarski",
    "Vanda Luengo",
    "Vincent Guigue",
    "Bruno Despres",
    "Frédéric Nataf",
    "Julien Brajard",
    "Sylvie Thiria",
    "Catherine Matias",
    "Charlotte Dion-Blanc",
    "Claire Boyer",
    "Gérard Biau",
    "Gregory Nuel",
    "Idris Kharroubi",
    "Maud Thomas",
    "Maxime Sangnier",
    "Olivier Lopez",
    "Sylvain Le-Corff",
    "Tabea Rebafka",
    "Benjamin Fuks",
    "Stéphane Mottelet",
    "Tien-Tuan Dao",
    "julien mozziconacci",
    "Nicolas Aunai",
    "Thierry Dufour",
    "Abdenour Hadid",
    "Benjamin Quost",
    "Bruno Toupance",
    "Dominique Lenne",
    "Evelyne Heyer",
    "Franz Manni",
    "Grace Younes",
    "Lama Tarsissi",
    "Marie-Hélène (Mylène) Masson",
    "Marie-Hélène Abel",
    "Nathalie Martial-Braz",
    "Nicolas Patin",
    "Philippe Bonnifait",
    "Philippe Boulanger",
    "Philippe Trigano",
    "Raed Abu Zitar",
    "Samuel F. Feng",
    "Sébastien Destercke",
    "Tanujit Chakraborty",
    "Yves Grandvalet",
    "Zoheir ABOURA"
]

# Initialisez une liste vide pour stocker les métadonnées des publications
publications = []

# URL de base de l'API HAL
base_url = "https://api.archives-ouvertes.fr/search/"

# Parcourez la liste des auteurs et récupérez leurs publications
for auteur in auteurs:
    params = {
        "q": f'authFullName_s:"{auteur}"',
        "fl": "authFullName_s,authIdHal_i,authIdHal_s,title_s,halId_s,producedDateY_i,doiId_s,abstract_s,uri_s,domain_s,keyword_s",
        "rows": 10000  # Augmentez le nombre de lignes si nécessaire
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        publications.extend(data["response"]["docs"])
    else:
        print(f"Erreur pour l'auteur {auteur}: {response.status_code} - {response.text}")

# Créez un DataFrame Pandas à partir des métadonnées des publications
column_order = ["authFullName_s", "authIdHal_i", "authIdHal_s", "title_s", "halId_s", "producedDateY_i", "doiId_s", "keyword_s", "abstract_s", "uri_s", "domain_s"]

df = pd.DataFrame(publications, columns= column_order)

# Sauvegardez le DataFrame dans un fichier CSV
#df.to_csv("publications_hal_scai_complet.csv", index=False)

# Affichez le nombre total de publications récupérées
print(f"Nombre total de publications récupérées : {len(df)}")

Nombre total de publications récupérées : 12388


In [26]:
# Conversion les listes en chaînes dans la colonne "title_s"
df["title_s"] = df["title_s"].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Suppression des doublons
df = df.drop_duplicates(subset="title_s")
dfp = df.dropna(subset=["abstract_s", "keyword_s"])
print(f"Publications restantes après nettoyage : {len(dfp)}")

Publications restantes après nettoyage : 4206


In [28]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

dfp = dfp.copy()
dfp['language'] = 'unknown'

def detect_language(text):
    try:
        return detect(text)
    except:
        return "UNKNOWN"

def lang(df):
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df.loc[i,'language'] = detect_language(str(df.loc[i,'abstract_s']))
    print("Détection de la langue terminée \n")
    return df

dfp = lang(dfp)

def counter(df):
    counts = df['language'].value_counts()
    print("Nombre d'articles anglais :")
    print(counts.get('en', 0))

    print("Nombre d'articles francais :")
    print(counts.get('fr', 0))

    print("Nombre d'articles allemand :")
    print(counts.get('de', 0))

    print("Index des articles avec langue inconnue")
    print(df[df['language'] == 'UNKNOWN'].index.tolist())
    return

counter(dfp)


Détection de la langue terminée 

Nombre d'articles anglais :
3547
Nombre d'articles francais :
654
Nombre d'articles allemand :
2
Index des articles avec langue inconnue
[]


In [43]:
# ID de l'auteur
author_id = "a5007814380"  

# URL de l'API
url = f"https://api.openalex.org/works?filter=author.id:{author_id}"

# Faire la requête à l'API
response = requests.get(url)

# Vérifier que la requête a réussi
if response.status_code == 200:
    # Conversion de la réponse en JSON
    data = json.loads(response.text)

    # Sélection les métadonnées précises
    selected_data = []
    
    for work in data['results']:
        
        author_ids = []
        if 'authorships' in work: 
            for authorship in work['authorships']:
                # Ajoutez l'identifiant de l'auteur à la liste 'author_ids'
                author_ids.append(authorship["author"]["id"])
                
        selected_data.append({
            'id' : work['id'],
            'title': work['title'],
            'authors_id' : author_ids, 
            'referenced_works': work['referenced_works'], 
            'related_works': work['related_works'], 
            'cited_by_api_url': work['cited_by_api_url'],    
        })

    # Conversion des données sélectionnées en DataFrame
    df = pd.DataFrame(selected_data)

    # Exportation du DataFrame en CSV
    df.to_csv('meta_pub_oa.csv', index=False)
else:
    print(f"Erreur : {response.status_code}")

In [ ]:
auteurs = [
    "a5063079937",
    "a5064456106",
    "a5075692514",
    "a5076657262",
    "a5060263799",
    "a5059667793",
    "a5010298238",
    "a5009962581",
    "a5019206372",
    "a5026117716",
    "Clément Mabi",
    "a5045653236", #David Flacher
    # pas là "M. Shawky",
    "a5054128778", #Serge Bouchardon 
    "a5083701333",
    #Bérangère Bihan-Avalle
    #Caroline Marti
    "a5043441322", #Laurent Petit
    "a5065665642", # ou  Pierre-Carl Langlais a5090355875 
    "a5044456322",
    "a5072965626",
    "a5050309898",
    "a5022276793",
    "a5035893506", #Guilhem Lavaux, y en a 2 donc un dechet
    #"Henry Joy McCracken", chelou
    "a5051397820",
    "a5075521248", # Yohan Dubois en x2
    "a5048168479",
    "a5008261497",
    "a5006372418",
    # existe pas "Slavica Jonic",
    "a5032837587",
    "a5082959305",
    "a5000035181", # y en a 2?? "Pierre-Yves Boëlle",
    "a5008914363", # y en a 2 "Renaud Piarroux",
    "a5012443434",
    "a5086118723", # Jean marc chouvel y en a 3
    "a5042745853",
    "a5085170922",
    "a5073978648",
    "a5060825472",
    "a5009145141", # 2 differents "Benjamin Piwowarski",
    "a5005922530", # 2 a cause du nom "Catherine Achard",
    "a5079026902", # 2 a cause du nom "Catherine Pelachaud",
    "a5019033794", # plusieurs "Gilles Bailly",
    "a5084974660",
    "a5015493311",
    "a5000942708", # 2 differents "Laure Soulier",
    "a5024969402",
    "a5022871131", # 2 differents Matthieu Cord
    "a5016381589",
    "a5049398785",
    "a5057665161", # 2 différents "Nathanaël Jarrassé",
    "a5008211251",
    "a5047587289",
    "a5017490804",
    "a5018812634",
    "a5042850624",
    "a5020385150", # 2 differents importants "Pascal Morin",
    "a5053141872", # 2 differents "Pierre Bessière",
    "a5016542830", # 2 differents "Sinan Haliyo",
    "a5003629424",
    "a5038174177", # 2 differents importants  "Alessandra Carbone" 
    "a5076453514", # 2 differents "Elodie Laine",
    "a5035771024",
    "a5049132515",
    "a5007007631",
    "a5089708177", # doute "Emmanuel Bertin",
    "a5064812526",
    "a5008102443", # 2 differents "Maryvonne Gerin",
    "a5061694871",
    "a5032663932",
    "a5074319502", # 2 differents " Philippe Zarka",
    "a5005420349",
    "a5055383240",
    "a5056834851", # 2 differents " Xavier Tannier",
    "a5044546919", # plusieurs differents importants  " Amal El Fallah-Seghrouchni " 
    "a5019342840",
    "a5069148908",
    "a5008504744",
    "a5057674250",
    "a5020410268",
    "a5040561209",
    "a5029270613",
    "a5003029415",
    "a5076504951", # 2 differents " Christophe Marsala",
    "a5008947610",
    "a5062847773", # 2 differents " Emmanuel Hyon "
    "a5040506059",
    "a5063171222", # 2 differents "Evripidis Bampis "
    "a5031467845",
    "a5091734149",
    "a5020645950", # 2 differents " Jean-Daniel Kant "
    "a5052738299", # 2 differents "Jean‐Gabriel Ganascia "
    "a5009688030",
    "a5056981393",
    "a5080217489", # 2 differents Maria Potop-Butucaru 
    "a5031952531", # 2 differents Matthieu Latapy  
    "a5000925369",
    "a5012670875",
    "a5025612115",
    "a5086752907",
    "a5073883755", # 2 differents Sébastien Tixeuil 
    "a5063338404", # 2 differents Spyros Angelopoulos 
    "a5049035512",
    "a5073711335",
    "a5044389669",
    "a5090273130",
    "a5004798802",
    "a5073300655", # 3 differents Julien Brajard
    "a5042022387", # 2 differents Sylvie Thiria 
    "a5082187550",
    "a5011096724",
    "a5010301554",
    "a5007814380",
    "a5087944743",
    "a5051885232",# 2 differents Idris Kharroubi 
    "a5076190399",# 2 differents Maud Thomas 
    "a5043113193",
    "a5041522495", # 3 differents importants Olivier Lopez
    "a5049226969",
    "a5011379612",
    "a5029271059",
    "a5043406295",
    "a5051516688", #  Tien Tuan Dao  pas de mention SU
    "a5027346778",
    "a5040717345",
    "a5048594819", # 2 differents AThierry Dufour 
    "a5013928164", # 2 differents Abdenour Hadid 
    "a5090882219",
    "a5024260775",
    "a5051963748",
    "a5005868901",
    "a5044064042", # 2 differents  Franz Manni  
    "a5027364763", # 2 differents Grace Younes 
    "a5033136511",
    "a5076437822",
    "a5064018319",
    "a5049226428",
    "a5000802564",
    "a5050418541", # 2 differents Philippe Bonnifait 
    "a5085744367",
    "a5054524844", # pas de mention de SU Philippe Trigano
    "a5078607983", # 3 differents  Raed Abu Zitar 
    "a5008158634",
    "a5070285963",
    "a5012926469",
    "a5021351429",
    "a5009983613"
]
